In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from use_case.baseline import * 
from tests.eval import *

np.random.seed(1337)

N = 10
payoff_i = np.random.uniform(-10, 10, (N, N))
payoff_j = np.random.uniform(-10, 10, (N, N))

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 1000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 1)

# Actual Run

In [3]:
from models.model import *
from models.trainer import *


In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

(18, 10) 9.42934343544164


In [ ]:
evaluate_policy(model, env, 10)


    Average Return: 0.0656039389081976
    Total returns: 0.656039389081976
    
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[103  69  83  45  43  65  40  49  66  38  58  41  39  43  33  49  35  37
  31  33]


: 

In [ ]:
# Setup the training loop
training_parameters = TrainingParameters(
    outer_loops = 200,
    hypernet_training_loops= 5, 
    actor_training_loops = 100,
    experience_buffer_size = 3,

    actor_learning_rate= 2.5e-4,
    critic_learning_rate = 2.5e-4,
    hypernet_learning_rate = 2.5e-4,
)

train_model(model, env, training_parameters)
        

Epoch 0


Hypernet Loop: 100%|██████████| 30/30 [00:07<00:00,  4.20it/s]



    Average Policy Loss 0.009680504910647869
    Average Entropy Loss: 36.09934616088867
    Average JSD Loss: 0.001468414906412363
    

    Average Return: 0.11158655686338168
    Total returns: 1.1158655686338168
    
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[132  91  74  79  67  54  28  48  43  44  38  35  52  37  33  24  28  37
  19  37]


Actor Training: 100%|██████████| 100/100 [00:44<00:00,  2.26it/s]



    Average Policy Loss: 0.03668736208463088
    Average Value Loss: 24.688346424102782
    Average Entropy Loss: 11.961390228271485 
    

    Average Return: 0.029856960693596103
    Total returns: 0.298569606935961
    
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[144  88  51  67  43  53  38  58  47  38  49  36  30  38  46  32  36  36
  33  37]
Epoch 1


...

Hypernet Loop: 100%|██████████| 30/30 [00:18<00:00,  1.66it/s]



    Average Policy Loss 0.005737268831580877
    Average Entropy Loss: 36.11208724975586
    Average JSD Loss: 0.000741052208468318
    

    Average Return: 0.022475401370138547
    Total returns: 0.22475401370138548
    
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[167 112  86  76  69  68  24  57  31  36  42  29  40  34  25  36  16  21
  18  13]


Actor Training: 100%|██████████| 100/100 [00:37<00:00,  2.65it/s]



    Average Policy Loss: 0.019926054645766272
    Average Value Loss: 24.528240098953248
    Average Entropy Loss: 11.971771671772004 
    

    Average Return: 0.038529306033813084
    Total returns: 0.3852930603381308
    
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[171  94  73  87  62  61  27  49  39  39  55  30  46  37  22  32  22  22
  19  13]
Epoch 8


Hypernet Loop:  43%|████▎     | 13/30 [00:02<00:03,  4.62it/s]